In [1]:
from sklearn.utils import shuffle
from keras.datasets import mnist
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential, load_model
from keras.layers import Dense, LeakyReLU,ZeroPadding2D, Dropout, Flatten, Conv2D, MaxPool2D, Activation, BatchNormalization, MaxPooling2D, Reshape, UpSampling2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import models
from collections import Counter
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras.datasets import mnist
from keras.models import load_model, Model
import numpy as np
from PIL import Image
import argparse
import math

Using TensorFlow backend.


### DCGAN

Le code du dcgan se trouve dans dcgan.py, pour l'executer :

python dcgan.py

les models (generateur et discrimant) se trouvent dans : models/gans/dcgan

les images généres par le GAN tout au long du training se trouvent dans : models/dcgan

### Recuperation du discriminateur

In [23]:
discriminator = load_model("models/gans/dcgan/discrimant.hdf5")

In [24]:
output = discriminator.get_layer("dropout_1").output
x = Dense(10, activation='softmax',name='logits')(output)

In [25]:
new_model = Model(discriminator.input, x)

In [26]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
__________

### Training

In [27]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images / 127.5 - 1.
test_images = test_images / 127.5 - 1.

train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

# train_labels = to_categorical(train_labels) 
test_labels = to_categorical(test_labels)


x, y = shuffle(train_images, train_labels, random_state=15)
x_train = x[0:100]
y_train = y[0:100].reshape(-1, 1)
y_train = to_categorical(y_train) 

# Sans data Augmentation

In [30]:
mcp = ModelCheckpoint("models/dcgan_best.h5", monitor='val_acc', verbose=0, save_best_only=True, mode='auto')
new_model.compile(loss='categorical_crossentropy', optimizer = "adam", metrics = ["accuracy"])

history = new_model.fit(x_train, y_train, validation_data=(test_images, test_labels), epochs=60, callbacks=[mcp])

Train on 100 samples, validate on 10000 samples
Epoch 1/60
100/100 [==============================] - 2s 19ms/step - loss: 0.0328 - acc: 0.9900 - val_loss: 0.2247 - val_acc: 0.9416
Epoch 2/60
100/100 [==============================] - 1s 6ms/step - loss: 0.0103 - acc: 1.0000 - val_loss: 0.5591 - val_acc: 0.8799
Epoch 3/60
100/100 [==============================] - 1s 7ms/step - loss: 0.0252 - acc: 0.9900 - val_loss: 0.2876 - val_acc: 0.9291
Epoch 4/60
100/100 [==============================] - 1s 6ms/step - loss: 0.0132 - acc: 1.0000 - val_loss: 0.4728 - val_acc: 0.8964
Epoch 5/60
100/100 [==============================] - 1s 7ms/step - loss: 0.1002 - acc: 0.9900 - val_loss: 0.3756 - val_acc: 0.9155
Epoch 6/60
100/100 [==============================] - 1s 6ms/step - loss: 0.0190 - acc: 0.9900 - val_loss: 0.5529 - val_acc: 0.8829
Epoch 7/60
100/100 [==============================] - 1s 6ms/step - loss: 0.0240 - acc: 0.9900 - val_loss: 0.6915 - val_acc: 0.8547
Epoch 8/60
100/100 [=======

### best accuracy

In [31]:
test_model = load_model("models/dcgan_best.h5")
test_loss, test_acc = test_model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 1s 104us/step


0.9416

# Avec data Augmentation

In [7]:
mcp = ModelCheckpoint("models/dcgan_aug_best.h5", monitor='val_acc', verbose=0, save_best_only=True, mode='auto')
new_model.compile(loss='categorical_crossentropy', optimizer = "adam", metrics = ["accuracy"])

In [8]:
datagen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1)  # randomly flip images

datagen.fit(x_train)

In [9]:
history2 = new_model.fit_generator(datagen.flow(x_train,y_train, batch_size=32),
                    epochs = 50, callbacks=[mcp],
                    validation_data = (test_images,test_labels),
                    verbose = 1, steps_per_epoch=20)

Epoch 1/50
20/20 [==============================] - 6s 275ms/step - loss: 2.2720 - acc: 0.2579 - val_loss: 2.0610 - val_acc: 0.4039
Epoch 2/50
20/20 [==============================] - 4s 185ms/step - loss: 1.4856 - acc: 0.4710 - val_loss: 1.4048 - val_acc: 0.5326
Epoch 3/50
20/20 [==============================] - 4s 181ms/step - loss: 1.0748 - acc: 0.6507 - val_loss: 1.2486 - val_acc: 0.5781
Epoch 4/50
20/20 [==============================] - 4s 192ms/step - loss: 0.8514 - acc: 0.7133 - val_loss: 0.7828 - val_acc: 0.7315
Epoch 5/50
20/20 [==============================] - 4s 187ms/step - loss: 0.7126 - acc: 0.7570 - val_loss: 0.6169 - val_acc: 0.7942
Epoch 6/50
20/20 [==============================] - 4s 184ms/step - loss: 0.6199 - acc: 0.7798 - val_loss: 0.4287 - val_acc: 0.8736
Epoch 7/50
20/20 [==============================] - 4s 180ms/step - loss: 0.3949 - acc: 0.8453 - val_loss: 0.2482 - val_acc: 0.9260
Epoch 8/50
20/20 [==============================] - 4s 181ms/step - loss: 0.

### best accuracy

In [10]:
test_model = load_model("models/dcgan_aug_best.h5")
test_loss, test_acc = test_model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 1s 66us/step


0.9547